# Лабораторна робота №3:Знайомство з нейромережами. Маринін Іван Павло Ігорович, ФІ-32мн

Датасет: https://www.kaggle.com/datasets/gabrielsantello/medical-malpractice-insurance-dataset

1. Повнозв'язані нейронні мережі

In [1]:
import pandas as pd

In [21]:
df = pd.read_csv('medicalmalpractice.csv', delimiter=',')
df = df[df.apply(lambda x: x.ne('Unknown').all(), axis=1)]
df.info()
df = df.drop(columns=['Specialty'])
df = pd.get_dummies(df, drop_first=True)
df.info()
df.to_csv('medicalmalpractice1.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55158 entries, 0 to 79209
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Amount            55158 non-null  int64 
 1   Severity          55158 non-null  int64 
 2   Age               55158 non-null  int64 
 3   Private Attorney  55158 non-null  int64 
 4   Marital Status    55158 non-null  int64 
 5   Specialty         55158 non-null  object
 6   Insurance         55158 non-null  object
 7   Gender            55158 non-null  object
dtypes: int64(5), object(3)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 55158 entries, 0 to 79209
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype
---  ------                          --------------  -----
 0   Amount                          55158 non-null  int64
 1   Severity                        55158 non-null  int64
 2   Age                             5

In [14]:
import csv
import numpy as np

fname = "medicalmalpractice1.csv"

all_features = []
all_targets = []
with open(fname) as f:
    print(type(f))
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

<class '_io.TextIOWrapper'>
HEADER: Amount,Severity,Age,Private Attorney,Marital Status,Insurance_No Insurance,Insurance_Private,Insurance_Workers Compensation,Gender_Male
EXAMPLE FEATURES: [57041.0, 7.0, 62.0, 1.0, 2.0, 0.0, 1.0, 0.0]
features.shape: (55158, 8)
targets.shape: (55158, 1)


In [15]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 44127
Number of validation samples: 11031


In [16]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 12772 (28.94% of total)


In [17]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

In [18]:
import keras

model = keras.Sequential(
    [
        keras.Input(shape=train_features.shape[1:]),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               2304      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 134145 (524.00 KB)
Trainable params: 134

In [19]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.keras")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Epoch 1/30

22/22 - 3s - loss: 2.8882e-05 - fn: 3761.0000 - fp: 13531.0000 - tn: 17824.0000 - tp: 9011.0000 - precision: 0.3997 - recall: 0.7055 - val_loss: 0.5509 - val_fn: 755.0000 - val_fp: 2619.0000 - val_tn: 5291.0000 - val_tp: 2366.0000 - val_precision: 0.4746 - val_recall: 0.7581 - 3s/epoch - 156ms/step
Epoch 2/30
22/22 - 1s - loss: 2.3305e-05 - fn: 2469.0000 - fp: 10194.0000 - tn: 21161.0000 - tp: 10303.0000 - precision: 0.5027 - recall: 0.8067 - val_loss: 0.4608 - val_fn: 425.0000 - val_fp: 2351.0000 - val_tn: 5559.0000 - val_tp: 2696.0000 - val_precision: 0.5342 - val_recall: 0.8638 - 912ms/epoch - 41ms/step
Epoch 3/30
22/22 - 1s - loss: 2.0394e-05 - fn: 2008.0000 - fp: 8634.0000 - tn: 22721.0000 - tp: 10764.0000 - precision: 0.5549 - recall: 0.8428 - val_loss: 0.4221 - val_fn: 440.0000 - val_fp: 1982.0000 - val_tn: 5928.0000 - val_tp: 2681.0000 - val_precision: 0.5750 - val_recall: 0.8590 - 912ms/epoch - 41ms/step
Epoch 4/30
22/22 - 1s - loss: 1.8396e-05 - fn: 1769.0000 - fp

2. Згорткові нейронні мережі

Датасет із зображеннями: https://www.kaggle.com/datasets/karimabdulnabi/fruit-classification10-class

In [105]:
from keras.preprocessing.image import ImageDataGenerator

train_data_path = "D:/Універ/магістратура/1 семестр/Data Analysis/lab 3/My_data/train"
test_data_path = "D:/Універ/магістратура/1 семестр/Data Analysis/lab 3/My_data//test"

img_size = (28, 28)
batch_size = 128


train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',  
    color_mode='grayscale'
)


test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',  
    color_mode='grayscale'
)

num_classes = len(train_generator.class_indices)


(x_train, y_train) = train_generator.next()
(x_test, y_test) = test_generator.next()

print("Shape of x_train:", x_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_test:", y_test.shape)

Found 2301 images belonging to 10 classes.
Found 1025 images belonging to 10 classes.
Shape of x_train: (128, 28, 28, 1)
Shape of y_train: (128,)
Shape of x_test: (128, 28, 28, 1)
Shape of y_test: (128,)


In [106]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
input_shape = (28, 28, 1)

In [107]:
from keras import layers

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(512, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_32 (MaxPooli  (None, 13, 13, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_33 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_33 (MaxPooli  (None, 5, 5, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_16 (Flatten)        (None, 1600)              0         
                                                                 
 dense_31 (Dense)            (None, 128)             

In [108]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/15
1/1 [==============================] - 2s 2s/step - loss: 2.3045 - accuracy: 0.0783 - val_loss: 2.3004 - val_accuracy: 0.0000e+00
Epoch 2/15
1/1 [==============================] - 0s 94ms/step - loss: 2.2966 - accuracy: 0.1478 - val_loss: 2.2957 - val_accuracy: 0.0769
Epoch 3/15
1/1 [==============================] - 0s 87ms/step - loss: 2.2858 - accuracy: 0.1565 - val_loss: 2.3105 - val_accuracy: 0.0769
Epoch 4/15
1/1 [==============================] - 0s 86ms/step - loss: 2.2765 - accuracy: 0.1391 - val_loss: 2.3322 - val_accuracy: 0.0769
Epoch 5/15
1/1 [==============================] - 0s 113ms/step - loss: 2.2567 - accuracy: 0.1391 - val_loss: 2.3503 - val_accuracy: 0.0769
Epoch 6/15
1/1 [==============================] - 0s 86ms/step - loss: 2.2518 - accuracy: 0.1478 - val_loss: 2.3352 - val_accuracy: 0.0769
Epoch 7/15
1/1 [==============================] - 0s 99ms/step - loss: 2.2442 - accuracy: 0.1478 - val_loss: 2.3176 - val_accuracy: 0.0769
Epoch 8/15
1/1 [========

Другий варіант завдання №2.

In [124]:
import os
import numpy as np
from zipfile import ZipFile
from keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array

def load_images_from_folder(folder_path):
    images = []
    labels = []
    class_labels = os.listdir(folder_path)
    
    for class_label in class_labels:
        class_path = os.path.join(folder_path, class_label)
        
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = load_img(img_path, target_size=(28, 28), color_mode='grayscale')
            img_array = img_to_array(img)
            img_array /= 255.0  
            images.append(img_array)
            labels.append(int(class_label)) 
    return np.array(images), np.array(labels)

train_folder_path = "D:/Універ/магістратура/1 семестр/Data Analysis/lab 3/My_data1/train"
x_train, y_train = load_images_from_folder(train_folder_path)

test_folder_path = "D:/Універ/магістратура/1 семестр/Data Analysis/lab 3/My_data1/test"
x_test, y_test = load_images_from_folder(test_folder_path)

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train-1, num_classes)
y_test = to_categorical(y_test-1, num_classes)

x_train shape: (2591, 28, 28, 1, 1)
y_train shape: (2591,)
2591 train samples
735 test samples


In [125]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dense(128, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(512, activation="relu"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_42 (MaxPooli  (None, 13, 13, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_43 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_43 (MaxPooli  (None, 5, 5, 64)          0         
 ng2D)                                                           
                                                                 
 dense_42 (Dense)            (None, 5, 5, 128)         8320      
                                                                 
 dense_43 (Dense)            (None, 5, 5, 256)       

In [126]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/15
19/19 [==============================] - 4s 125ms/step - loss: 2.2157 - accuracy: 0.1283 - val_loss: 7.5860 - val_accuracy: 0.0038
Epoch 2/15
19/19 [==============================] - 2s 107ms/step - loss: 2.1343 - accuracy: 0.1832 - val_loss: 6.3321 - val_accuracy: 0.0000e+00
Epoch 3/15
19/19 [==============================] - 2s 110ms/step - loss: 2.0433 - accuracy: 0.2312 - val_loss: 11.2716 - val_accuracy: 0.0000e+00
Epoch 4/15
19/19 [==============================] - 2s 106ms/step - loss: 1.9481 - accuracy: 0.2879 - val_loss: 12.8024 - val_accuracy: 0.0000e+00
Epoch 5/15
19/19 [==============================] - 2s 106ms/step - loss: 1.8528 - accuracy: 0.3299 - val_loss: 12.5501 - val_accuracy: 0.0000e+00
Epoch 6/15
19/19 [==============================] - 2s 103ms/step - loss: 1.8134 - accuracy: 0.3316 - val_loss: 11.4474 - val_accuracy: 0.0000e+00
Epoch 7/15
19/19 [==============================] - 2s 113ms/step - loss: 1.7435 - accuracy: 0.3672 - val_loss: 15.6576 - va

3. Рекурентні нейронні мережі

In [127]:
max_features = 20000 
maxlen = 200

In [128]:
inputs = keras.Input(shape=(None,), dtype="int32")

x = layers.Embedding(max_features, 128)(inputs)

x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         2560000   
                                                                 
 bidirectional (Bidirection  (None, None, 128)         98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_46 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2757761 (10.52 MB)
Trainable params: 2757761 (1

In [151]:
from keras.preprocessing.sequence import pad_sequences

df = pd.read_csv('Mental Health Dataset.csv')
df = df.dropna(subset=['posts'])

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

tokenizer = keras.preprocessing.text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(train_df["posts"])

x_train = tokenizer.texts_to_sequences(train_df["posts"])
x_val = tokenizer.texts_to_sequences(val_df["posts"])

x_train = pad_sequences(x_train, maxlen=maxlen)
x_val = pad_sequences(x_val, maxlen=maxlen)

y_train = train_df["intensity"].values
y_val = val_df["intensity"].values

print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")

8312 Training sequences
2079 Validation sequences


In [152]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
260/260 [==============================] - 86s 299ms/step - loss: -15.8637 - accuracy: 0.4182 - val_loss: -26.8060 - val_accuracy: 0.4276
Epoch 2/2
260/260 [==============================] - 73s 281ms/step - loss: -34.6036 - accuracy: 0.4193 - val_loss: -45.1671 - val_accuracy: 0.4276
